In [ ]:
import matplotlib.pyplot as plt
import sys
from keras.preprocessing.image import array_to_img,img_to_array,load_img
import numpy as np
import os
from keras.preprocessing import image
import keras
from sklearn.model_selection import train_test_split

import load_image
#import Train_metric_cnn
#import Faster_Grad_CAM
#import Train_metric_cnn

#GPUを使用するためのコード  
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="0", # specify GPU number
        allow_growth=True
    )
)
set_session(tf.Session(config=config))

#ここを変えてから動かす
#クラスはここに記入
#MODEL_NAMEは最後尾に.h5がないとだめ
CLASS = 7
EPOCH = 4
MODEL_NAME = "model_label_7_100.json"
MODEL_WEIGHTS = "model_label_7_100.h5"
PATH = "model/"



#データセットの作成
x_train_0 = load_image.load_images("images/data/0/")
x_train_1 = load_image.load_images("images/data/1/")
x_train_2 = load_image.load_images("images/data/2/")
x_train_3 = load_image.load_images("images/data/3/")
x_train_4 = load_image.load_images("images/data/4/")
x_train_5 = load_image.load_images("images/data/5/")
x_train_6 = load_image.load_images("images/data/6/")
#x_train_7 = load_images("images/data/7/")
print(x_train_0)

x_train_0 /= 255
x_train_1 /= 255
x_train_2 /= 255
x_train_3 /= 255
x_train_4 /= 255
x_train_5 /= 255
x_train_6 /= 255
#x_train_7 /= 255

train_label_0 = np.full(len(x_train_0),0)
train_label_1 = np.full(len(x_train_1),1)
train_label_2 = np.full(len(x_train_2),2)
train_label_3 = np.full(len(x_train_3),3)
train_label_4 = np.full(len(x_train_4),4)
train_label_5 = np.full(len(x_train_5),5)
train_label_6 = np.full(len(x_train_6),6)
#train_label_7 = np.full(len(x_train_7),7)


x_train_0, x_test_0, y_train_0, y_test_0 = train_test_split(x_train_0, train_label_0, train_size=0.8)
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(x_train_1, train_label_1, train_size=0.8)
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(x_train_2, train_label_2, train_size=0.8)
x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_train_3, train_label_3, train_size=0.8)
x_train_4, x_test_4, y_train_4, y_test_4 = train_test_split(x_train_4, train_label_4, train_size=0.8)
x_train_5, x_test_5, y_train_5, y_test_5 = train_test_split(x_train_5, train_label_5, train_size=0.8)
x_train_6, x_test_6, y_train_6, y_test_6 = train_test_split(x_train_6, train_label_6, train_size=0.8)
#x_train_7, x_test_7, y_train_7, y_test_7 = train_test_split(x_train_7, train_label_7, train_size=0.8)

X_train = np.vstack((x_train_0,x_train_1,x_train_2,x_train_3,x_train_4,x_train_5,x_train_6))
X_test = np.vstack((x_test_0,x_test_1,x_test_2,x_test_3,x_test_4,x_test_5,x_test_6))
y_train = np.hstack((y_train_0,y_train_1,y_train_2,y_train_3,y_train_4,y_train_5,y_train_6))
y_test = np.hstack((y_test_0,y_test_1,y_test_2,y_test_3,y_test_4,y_test_5,y_test_6))

Y_train = keras.utils.to_categorical(y_train)
Y_test = keras.utils.to_categorical(y_test)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
from keras.applications import vgg19
from keras.engine.topology import Layer
from keras import backend as K
from keras.layers import Input, Activation
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras
from sklearn import metrics

def train_mobileV2(X_train, Y_train, X_test, Y_test, epoch, classes=2, alpha_=0.5):
    mobile =vgg19.VGG19(include_top=True,input_shape=(224,224,3),weights='imagenet')
    
    # 最終層削除
    mobile.layers.pop()
    v2 = Model(inputs=mobile.input,outputs=mobile.layers[-1].output)

    model = build_arcface(X_train, classes, v2)

    datagen = ImageDataGenerator(rotation_range=20,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 horizontal_flip=True)
    
    datagen.fit(X_train)

    #cnnの学習
    hist = model.fit_generator(datagen.flow([X_train, Y_train], Y_train, batch_size=32),
                               steps_per_epoch=X_train.shape[0] /32,
                               validation_data=([X_test, Y_test], Y_test),
                               epochs=epoch, 
                               verbose=1)

    plt.figure()               
    plt.plot(hist.history['val_acc'],label="val_acc")
    plt.legend(loc="lower right")
    plt.show()

    return model

# mobilenetV2と接合して学習
def build_arcface(x, classes, base_model):
    #add new layers 
    hidden = base_model.output
    yinput = Input(shape=(classes,)) #ArcFaceで使用
    # stock hidden model
    c = Arcfacelayer(classes, 30, 0.05)([hidden,yinput]) #outputをクラス数と同じ数に
    prediction = Activation('softmax')(c)
    model = Model(inputs=[base_model.input, yinput], outputs=prediction)

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.0001, amsgrad=True),
                  metrics=['accuracy'])

    return model

#arcfaceの層
class Arcfacelayer(Layer):
    # s:softmaxの温度パラメータ, m:margin
    def __init__(self, output_dim, s=30, m=0.50, easy_margin=False):
        self.output_dim = output_dim
        self.s = s
        self.m = m
        self.easy_margin = easy_margin
        super(Arcfacelayer, self).__init__()

    # 重みの作成
    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[0][1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(Arcfacelayer, self).build(input_shape)

    # mainの処理 
    def call(self, x):
        y = x[1]
        x_normalize = tf.math.l2_normalize(x[0]) # x = x'/ ||x'||2
        k_normalize = tf.math.l2_normalize(self.kernel) # Wj = Wj' / ||Wj'||2

        cos_m = K.cos(self.m)
        sin_m = K.sin(self.m)
        th = K.cos(np.pi - self.m)
        mm = K.sin(np.pi - self.m) * self.m

        cosine = K.dot(x_normalize, k_normalize) # W.Txの内積
        sine = K.sqrt(1.0 - K.square(cosine))

        phi = cosine * cos_m - sine * sin_m #cos(θ+m)の加法定理

        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine) 

        else:
            phi = tf.where(cosine > th, phi, cosine - mm) 

        # 正解クラス:cos(θ+m) 他のクラス:cosθ 
        output = (y * phi) + ((1.0 - y) * cosine) 
        output *= self.s

        return output

    def compute_output_shape(self, input_shape):

        return (input_shape[0][0], self.output_dim) #入力[x,y]のためx[0]はinput_shape[0][0]

In [ ]:
import time

def GradCam(model, x, layer_name, class_, top50):
    X = np.expand_dims(x,axis=0)
    
    # 前処理
    target = np.array([0, 1]).reshape((1,-1))
    class_idx = class_
    class_output = model.output[:, class_idx]
    
    # 勾配を取得
    before = time.time()
    conv_output = model.get_layer(layer_name).output   # layer_nameのレイヤーのアウトプット
    grads = K.gradients(class_output, conv_output)[0]  # gradients(loss, variables) で、variablesのlossに関しての勾配を返す
    gradient_function = K.function([model.input[0],model.input[1]], [conv_output, grads])  # model.inputを入力すると、conv_outputとgradsを出力する関数
    
    output, grads_val = gradient_function([X, target])
    output, grads_val = output[0], grads_val[0]

    # 重みを平均化して、レイヤーのアウトプットに乗じる
    weights = np.mean(grads_val, axis=(0, 1))
    if top50 == True:
        label = np.argsort(weights)
        cam = np.dot(output[:,:,label[-50:]], weights[label[-50:]])
    else:
        cam = np.dot(output, weights)

    # ヒートマップにして合成
    cam = cv2.resize(cam, (x.shape[1], x.shape[0]), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    
    jetcam = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)  # モノクロ画像に疑似的に色をつける
    jetcam = cv2.cvtColor(jetcam, cv2.COLOR_BGR2RGB)  # 色をRGBに変換
    jetcam = (np.float32(jetcam) + x*255 / 2)   # もとの画像に合成

    return jetcam, weights, time.time()-before

In [ ]:
from sklearn.cluster import KMeans

# compare grad and faster-grad
def show_result(model, model_small, layer_name, data, no, kmeans, channel_weight, channel_adress, top50=False):
    original, result_grad, result_faster, time0, time1 = [], [], [], [], []
    for i in range(5):
        original.append(data[no[i]]) 
        img0, _, time_0 = GradCam(model, data[no[i]], layer_name, 1, top50)
        img1, time_1 = predict_faster_gradcam(data[no[i]], model_small, kmeans, channel_weight, channel_adress)
        result_grad.append(img0)
        result_faster.append(img1)
        time0.append(time_0)
        time1.append(time_1)

    plt.figure(figsize=(15,10))
    for i in range(5):
        plt.subplot(3,5,i+1)
        plt.axis("off")
        if i == 0:
            plt.title("original")
        plt.imshow(original[i])
    for i in range(5):
        plt.subplot(3,5,i+6)
        plt.axis("off")
        if i == 0:
            time_ = int(np.mean(time1)*1000)
            plt.title("Faster-Grad-CAM \n(%d msec)" % time_)
        plt.imshow(array_to_img(result_faster[i]))
    for i in range(5):
        plt.subplot(3,5,i+11)
        plt.axis("off")
        if i == 0:
            time_ = int(np.mean(time0)*1000)
            plt.title("Grad-CAM \n(%d msec)" % time_)
        plt.imshow(array_to_img(result_grad[i]))
    plt.show()

def train_faster_gradcam(x_normal, x_anomaly, model, clusters=10):
    # Arcfaceを削除
    model_embed = Model(model.get_layer(index=0).input, [model.layers[-13].get_output_at(-1), model.layers[-4].get_output_at(-1)])

    # pa class data
    _, vector_normal = model_embed.predict(x_normal)

    # gu class data
    _, vector_anomaly = model_embed.predict(x_anomaly)# shape[(len(x), 3, 3, 480), (len(x), 1280)]

    # k-means
    kmeans = KMeans(n_clusters=clusters, random_state=0).fit(vector_anomaly)
    labels = kmeans.labels_

    # channel database
    channel_weight, channel_adress = [], []
    temp_weight = np.zeros((clusters, 480))# 480="block_16_expand_relu".output
    print("Making Database...")
    for i in range(len(labels)):
        # x_anomalyについて一個ずつ重みを加算していく
        _, weight, _ = GradCam(model, x_anomaly[i], "block_16_expand_relu", 1, False)
        temp_weight[labels[i]] += weight #要確認
        print(i+1,"/",len(labels))

    for i in range(clusters):
        number = np.where(labels == i, 1, 0) #クラスタ内の個数
        average_weight = temp_weight[i] / np.sum(number) #重みの平均
        weight_adress = np.argsort(average_weight)
        channel_adress.append(weight_adress[-50:])
        channel_weight.append(average_weight[weight_adress[-50:]])

    return model_embed, kmeans, np.array(channel_weight), np.array(channel_adress), vector_normal

def predict_faster_gradcam(x, model, kmeans, channel_weight, channel_adress):
    before = time.time()
    channel_out, vector = model.predict(np.expand_dims(x, axis=0))
    channel_out = channel_out[0]
    cluster_no = kmeans.predict(vector)
    # レイヤーのアウトプットに乗じる
    cam = np.dot(channel_out[:,:,channel_adress[cluster_no][0]], channel_weight[cluster_no][0])

    # ヒートマップにして合成
    cam = cv2.resize(cam, (x.shape[1], x.shape[0]), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    
    jetcam = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)  # モノクロ画像に疑似的に色をつける
    jetcam = cv2.cvtColor(jetcam, cv2.COLOR_BGR2RGB)  # 色をRGBに変換
    jetcam = (np.float32(jetcam) + x*255 / 2)   # もとの画像に合成

    return jetcam, time.time()-before


In [ ]:
model = train_mobileV2(X_train, Y_train, X_test, Y_test, EPOCH, classes=7)